In [ ]:
import os
import time
from datetime import datetime, timedelta
import glob
from multiprocessing import Pool

from tqdm.notebook import tqdm, trange
import numpy as np
import matplotlib.pyplot as plt

from lmsiage.mesh_file import MeshFile

In [ ]:
#lag_dir = 'zarr'
mesh_dir = 'zarr/mesh'
age_dir = 'zarr/age'
MAX_LEN_FRACTIONS = 6

age_files = sorted(glob.glob(f'{age_dir}/*/age_*.zip'))
start_file = f'{age_dir}/1991/age_19910915.zip'
not_myi_files = [f for f in age_files if '0915.zip' not in f]
if len(not_myi_files) > 0:
    start_file = not_myi_files[-1]
start_date = datetime.strptime(os.path.basename(start_file), 'age_%Y%m%d.zip')
duration = 1 # years
end_date = datetime(start_date.year + duration, 12, 31)
max_i = (end_date - start_date).days
print(start_file, start_date, end_date, max_i)

In [ ]:
for i in trange(max_i):
    src_file_date = start_date + timedelta(i)
    dst_file_date = start_date + timedelta(i + 1)
    
    # load previously propagated MYI
    src_age_path = src_file_date.strftime(f'{age_dir}/%Y/age_%Y%m%d.zip')
    src_age = MeshFile(src_age_path)
    src_names = src_age.read_names()
    # select only N most recent fractions, N = MAX_LEN_FRACTIONS
    myi_names = sorted([n for n in src_names if n.startswith('sic') and len(n) > 3], reverse=True)[:MAX_LEN_FRACTIONS][::-1]
    src_myi = src_age.load(myi_names)
    
    # load mesh
    dst_mesh_path = dst_file_date.strftime(f'{mesh_dir}/%Y/mesh_%Y%m%d.zip')
    dst_mesh = MeshFile(dst_mesh_path)
    src2dst, w, ar, sic = dst_mesh.load(['src2dst', 'weights', 'ar', 'sic'], as_dict=False)
    ar[ar == 0] = 0.01

    # propagate all previous MYI
    dst_myi = {}
    for name, c0 in src_myi.items():
        c1 = np.zeros(src2dst[:,1].max()+1)
        np.add.at(c1, src2dst[:,1], c0[src2dst[:,0]] * w)
        c1 /= ar
        c1 = np.clip(c1, 0, 100)
        c1 = np.where(c1 > sic, sic, c1)
        dst_myi[name] = c1
    
    # save next MYI
    dst_age_path = dst_file_date.strftime(f'{age_dir}/%Y/age_%Y%m%d.zip')        
    dst_mf = MeshFile(dst_age_path)
    dst_mf.save(dst_myi, mode='o')
